In [1]:
import numpy as np
import pandas as pd
import random
import pickle
import itertools
import folium #mapping
import seaborn as sns #color palette
#from colormap import rgb2hex #chnage RGB codes to hex codes

In [2]:
## USE THIS CODE TO IMPORT DATA

file_name = 'data/results/results_toydata_uniform.pkl'
open_file = open(file_name, "rb")
loaded_list = pickle.load(open_file)
open_file.close()

##[(supplier_lat, supplier_lon), (receiver_lat, receiver_lon), distance, volume_transferred]
#loaded_list[0:5]

In [21]:
#convert to dataframe

df = pd.DataFrame(loaded_list, columns = ['supplier', 'receiver', 'distance', 'volume'])
df.head()
#df.shape #109031, 4

#list-ify the suppliers to get rid of weird numpy object datatype
supplier_split = df.supplier.to_list()
receiver_split = df.receiver.to_list()
#supplier_split[0:5]


#convert suppliers to lat and lon
supplier_lat= [supplier_split[j][0] for j in range(0,len(supplier_split))]
supplier_lon = [supplier_split[j][1] for j in range(0,len(supplier_split))]

#convert to list of receiver lat and long
receiver_lat= [receiver_split[j][0] for j in range(0,len(receiver_split))]
receiver_lon = [receiver_split[j][1] for j in range(0,len(receiver_split))]

#add back into the df
df['supplier_lat'] = supplier_lat
df['supplier_lon'] = supplier_lon
df['receiver_lat'] = receiver_lat
df['receiver_lon'] = receiver_lat

df.head()

,supplier,receiver,distance,volume,supplier_lat,supplier_lon,receiver_lat,receiver_lon
0,"[36.92375, -120.10412]","[36.20321655, -120.26839447]",50.516378,500.0,36.92375,-120.10412,36.203217,36.203217
1,"[36.92375, -120.10412]","[36.18690109, -120.26811218]",51.620549,500.0,36.92375,-120.10412,36.186901,36.186901
2,"[36.92375, -120.10412]","[36.20140457, -120.26834869]",50.638816,500.0,36.92375,-120.10412,36.201405,36.201405
3,"[36.92375, -120.10412]","[36.19958878, -120.26831055]",50.761602,500.0,36.92375,-120.10412,36.199589,36.199589
4,"[36.92375, -120.10412]","[36.19777679, -120.2682724]",50.884141,500.0,36.92375,-120.10412,36.197777,36.197777


In [72]:

# #create a color paltte
# palette = sns.color_palette(None, 10)
# palette

# palette[0]
# palette = palette.as_hex()
# palette[0]
# palette = [i.replace('#','') for i in palette]
# palette

#color palette
palette = ['red', 'green', 'purple', 'orange',  'darkblue',
           'gray','darkgreen', 'cadetblue', 'darkpurple', 'white', 'pink', 'lightblue']#, ‘lightgreen’] 
  #         ‘gray’, ‘black’, ‘lightgray’] 'darkred','lightred','blue','beige','white',

palette

['red',
 'green',
 'purple',
 'orange',
 'darkblue',
 'gray',
 'darkgreen',
 'cadetblue',
 'darkpurple',
 'white',
 'pink',
 'lightblue']

## With sort

In [73]:
df.shape

(119548, 8)

In [74]:
#just one
df[df['supplier_lat'] == 35.399380]

,supplier,receiver,distance,volume,supplier_lat,supplier_lon,receiver_lat,receiver_lon
63659,"[35.39938, -119.40813]","[35.3040123, -118.83643341]",32.951948,194.4,35.39938,-119.40813,35.304012,35.304012


In [75]:
df_counts = df.groupby(['supplier_lat','supplier_lon']).count().reset_index()
df_sorted = df_counts.sort_values('distance')
df_sorted.head(20)

,supplier_lat,supplier_lon,supplier,receiver,distance,volume,receiver_lat,receiver_lon
221,37.633454,-122.397961,1,1,1,1,1,1
190,37.074790,-120.877445,1,1,1,1,1,1
188,37.000100,-121.536700,1,1,1,1,1,1
50,33.894790,-118.164740,1,1,1,1,1,1
183,36.850693,-120.644469,1,1,1,1,1,1
176,36.685080,-119.552270,1,1,1,1,1,1
104,34.600471,-120.454797,1,1,1,1,1,1
24,33.615700,-114.686500,1,1,1,1,1,1
112,34.902694,-117.160594,1,1,1,1,1,1
194,37.114650,-120.943110,1,1,1,1,1,1


In [76]:
df_grouped = df.groupby(['supplier_lat','supplier_lon']).mean('distance').reset_index()
df_grouped.shape 
df_grouped = df_grouped.sort_values('distance')
df_grouped = df_grouped.head(5).reset_index()
df_grouped

df_small = pd.DataFrame()

#loop through and get the full dataset. can't group by the lat/long because it's some unhashable type.
for i in range(0,df_grouped.shape[0]):
    this_lat = df_grouped['supplier_lat'][i]
    this_lon = df_grouped['supplier_lon'][i]
    receivers = df[(df['supplier_lat'] == this_lat) & (df['supplier_lon'] == this_lon)]
    df_small = df_small.append(receivers)
    
df_small.shape
df_small

,supplier,receiver,distance,volume,supplier_lat,supplier_lon,receiver_lat,receiver_lon
1776,"[35.2097, -119.585]","[35.2057991, -119.60077667]",0.932273,500.0,35.20970,-119.585000,35.205799,35.205799
1777,"[35.2097, -119.585]","[35.20614243, -119.60223389]",1.005462,500.0,35.20970,-119.585000,35.206142,35.206142
1778,"[35.2097, -119.585]","[35.20867157, -119.60144043]",0.932883,500.0,35.20970,-119.585000,35.208672,35.208672
1779,"[35.2097, -119.585]","[35.26708221, -119.59991455]",4.044755,500.0,35.20970,-119.585000,35.267082,35.267082
1780,"[35.2097, -119.585]","[35.2671051, -119.59921265]",4.038209,500.0,35.20970,-119.585000,35.267105,35.267105
...,...,...,...,...,...,...,...,...
77108,"[35.45306, -119.719337]","[35.28014374, -118.88262939]",48.729032,500.0,35.45306,-119.719337,35.280144,35.280144
77109,"[35.45306, -119.719337]","[33.62336349, -117.932724]",162.140947,500.0,35.45306,-119.719337,33.623363,33.623363
77110,"[35.45306, -119.719337]","[33.6319313, -117.93341064]",161.654155,500.0,35.45306,-119.719337,33.631931,33.631931
77111,"[35.45306, -119.719337]","[33.64023209, -117.93100739]",161.293794,500.0,35.45306,-119.719337,33.640232,33.640232


In [77]:
#find unique suppliers for looping and filtering
unique_suppliers = df_small[['supplier_lat', 'supplier_lon']]
unique_suppliers = unique_suppliers.drop_duplicates()
unique_suppliers =unique_suppliers.reset_index(drop=True)
unique_suppliers
unique_suppliers['supplier_lon'][0]

-119.585

In [78]:
mapit = folium.Map( location=[39.4598465 , -121.94542694], zoom_start=20 )
receiver_count = 0
 
#plot all the suppliers 
latlon = df_small['supplier']
mapit = folium.Map( location=[39.4598465 , -121.94542694], zoom_start=5 )
#for coord in latlon:
#    folium.CircleMarker( location=[ coord[0], coord[1] ], fill=True,color='black', fill_color='black',radius=5 ).add_to( mapit ) #'

#find unique suppliers for looping and filtering
unique_suppliers = df_small[['supplier_lat', 'supplier_lon']]
unique_suppliers = unique_suppliers.drop_duplicates()
unique_suppliers =unique_suppliers.reset_index(drop=True)
unique_suppliers

#loop through suppliers, paint all the receivers in the same color
for i in range(0,(unique_suppliers.shape[0])):
    
    #big circle for supplier
    folium.CircleMarker( location=[ unique_suppliers['supplier_lat'][i],unique_suppliers['supplier_lon'][i] ], fill=True, fill_color=palette[i], color=palette[i],  radius=5 ).add_to( mapit ) #'
    
    #find all receivers associated with the supplier late and lon, filter the df
    this_lat = unique_suppliers['supplier_lat'][i]
    this_lon = unique_suppliers['supplier_lon'][i]
    receivers = df_small[(df_small['supplier_lat'] == this_lat) & (df_small['supplier_lon'] == this_lon)]
    
    #plot the receivers with a new color each time
    latlon = receivers['receiver']
    #print(receivers.shape[0])
    #print(receivers.shape[1])
    receiver_count += receivers.shape[0] #verify 1000
    
    #for each receiver  
    for coord in latlon: 
        
        #line coordinates
        line_coords = [ [unique_suppliers['supplier_lat'][i],unique_suppliers['supplier_lon'][i]],
                        [coord[0], coord[1] ]  ]
        my_PolyLine= folium.PolyLine(locations=line_coords,color=palette[i],weight=1)
        mapit.add_child(my_PolyLine)
        
        #dot for the receiver
        folium.CircleMarker( location=[ coord[0], coord[1] ], fill=True, fill_color=palette[i], color=palette[i],  radius=1 ).add_to( mapit )

print(receiver_count)
mapit

728


## Without sort

In [4]:
# 
#subset data for initial map
#df_small = df.head(1000)

In [6]:
# df_small

,supplier,receiver,distance,volume,supplier_lat,supplier_lon,receiver_lat,receiver_lon
0,"[36.92375, -120.10412]","[36.20321655, -120.26839447]",50.516378,500.0,36.92375,-120.10412,36.203217,36.203217
1,"[36.92375, -120.10412]","[36.18690109, -120.26811218]",51.620549,500.0,36.92375,-120.10412,36.186901,36.186901
2,"[36.92375, -120.10412]","[36.20140457, -120.26834869]",50.638816,500.0,36.92375,-120.10412,36.201405,36.201405
3,"[36.92375, -120.10412]","[36.19958878, -120.26831055]",50.761602,500.0,36.92375,-120.10412,36.199589,36.199589
4,"[36.92375, -120.10412]","[36.19777679, -120.2682724]",50.884141,500.0,36.92375,-120.10412,36.197777,36.197777
...,...,...,...,...,...,...,...,...
995,"[34.5953, -117.3647]","[33.676548, -118.02750397]",73.845109,500.0,34.59530,-117.36470,33.676548,33.676548
996,"[34.5953, -117.3647]","[33.669384, -118.01847076]",74.006286,500.0,34.59530,-117.36470,33.669384,33.669384
997,"[34.5953, -117.3647]","[33.67645264, -118.02755737]",73.852330,500.0,34.59530,-117.36470,33.676453,33.676453
998,"[34.5953, -117.3647]","[33.67001724, -118.01934052]",73.993849,500.0,34.59530,-117.36470,33.670017,33.670017


In [7]:
#find unique suppliers for looping and filtering
unique_suppliers = df_small[['supplier_lat', 'supplier_lon']]
unique_suppliers = unique_suppliers.drop_duplicates()
# unique_suppliers =unique_suppliers.reset_index(drop=True)
unique_suppliers
unique_suppliers['supplier_lon'][0]

-120.10412

In [8]:
# mapit = folium.Map( location=[39.4598465 , -121.94542694], zoom_start=20 )
# receiver_count = 0
 
# #plot all the suppliers 
# latlon = df_small['supplier']
# mapit = folium.Map( location=[39.4598465 , -121.94542694], zoom_start=5 )
# #for coord in latlon:
# #    folium.CircleMarker( location=[ coord[0], coord[1] ], fill=True,color='black', fill_color='black',radius=5 ).add_to( mapit ) #'

# #find unique suppliers for looping and filtering
# unique_suppliers = df_small[['supplier_lat', 'supplier_lon']]
# unique_suppliers = unique_suppliers.drop_duplicates()
# unique_suppliers =unique_suppliers.reset_index(drop=True)
# unique_suppliers

# #loop through suppliers, paint all the receivers in the same color
# for i in range(0,(unique_suppliers.shape[0])):
    
#     #big circle for supplier
#     folium.CircleMarker( location=[ unique_suppliers['supplier_lat'][i],unique_suppliers['supplier_lon'][i] ], fill=True, fill_color=palette[i], color=palette[i],  radius=5 ).add_to( mapit ) #'
    
#     #find all receivers associated with the supplier late and lon, filter the df
#     this_lat = unique_suppliers['supplier_lat'][i]
#     this_lon = unique_suppliers['supplier_lon'][i]
#     receivers = df_small[(df_small['supplier_lat'] == this_lat) & (df_small['supplier_lon'] == this_lon)]
    
#     #plot the receivers with a new color each time
#     latlon = receivers['receiver']
#     #print(receivers.shape[0])
#     #print(receivers.shape[1])
#     receiver_count += receivers.shape[0] #verify 1000
    
#     #for each receiver  
#     for coord in latlon: 
        
#         #line coordinates
#         line_coords = [ [unique_suppliers['supplier_lat'][i],unique_suppliers['supplier_lon'][i]],
#                         [coord[0], coord[1] ]  ]
#         my_PolyLine= folium.PolyLine(locations=line_coords,color=palette[i],weight=1)
#         mapit.add_child(my_PolyLine)
        
#         #dot for the receiver
#         folium.CircleMarker( location=[ coord[0], coord[1] ], fill=True, fill_color=palette[i], color=palette[i],  radius=1 ).add_to( mapit )

# print(receiver_count)
# mapit

1000


In [9]:
# #latlon = records[0:1000] #[ (51.249443914705175, -0.13878830247011467), (51.249443914705175, -0.13878830247011467), (51.249768239976866, -2.8610415615063034)]
# latlon = df_small['receiver']
# mapit = folium.Map( location=[39.4598465 , -121.94542694], zoom_start=6 )
# for coord in latlon:
#     folium.Marker( location=[ coord[0], coord[1] ], fill_color='#43d9de', radius=8 ).add_to( mapit )
    
# mapit

In [10]:
# import folium

# # Coordinates are 10 points on the great circle from Boston to
# # San Francisco.
# # Reference: http://williams.best.vwh.net/avform.htm#Intermediate
# #coordinates = [
#    # [42.3581, -71.0636],
#     #[42.82995815, -74.78991444],
#     #[43.17929819, -78.56603306],
#     #[43.40320216, -82.37774519],
#     #[43.49975489, -86.20965845],
#     #[41.4338549, -108.74485069],
#     #[40.67471747, -112.29609954],
#     #[39.8093434, -115.76190821],
#     #[38.84352776, -119.13665678],
#   #  [37.7833, -122.4167]]

# # line_coords = [ [unique_suppliers['supplier_lat'][0],unique_suppliers['supplier_lon'][0]],
#                         [coord[0], coord[1] ]  ]

# m.add_child(my_PolyLine)

# # Create the map and add the line
# m = folium.Map(location=[41.9, -97.3], zoom_start=4)
# my_PolyLine=folium.PolyLine(locations=line_coords,weight=5) #coordinates
# m.add_child(my_PolyLine)
# # m.save('line_example_newer.html')

NameError: name 'm' is not defined